In [21]:
import pandas as pd
from sqlalchemy import create_engine
import openpyxl
from math import isnan

## Extract CSVs and Excel Sheets into DataFrames

#### 1. Extract game.csv for AFL Game Results

In [22]:
game_file = "Resources/games.csv"
game_df = pd.read_csv(game_file)
game_df.head()

,gameId,year,round,date,venue,startTime,attendance,homeTeam,homeTeamScore,awayTeam,awayTeamScore,rainfall
0,2021R101,2021,R1,18-Mar-2021,M.C.G.,7:25 PM,49218,Richmond,105,Carlton,80,0.0
1,2021R102,2021,R1,19-Mar-2021,M.C.G.,7:50 PM,46051,Collingwood,53,Western Bulldogs,69,0.0
2,2021R103,2021,R1,20-Mar-2021,M.C.G.,1:45 PM,21365,Melbourne,80,Fremantle,58,0.0
3,2021R104,2021,R1,20-Mar-2021,Adelaide Oval,4:05 PM,26985,Adelaide,103,Geelong,91,0.0
4,2021R105,2021,R1,20-Mar-2021,Docklands,7:25 PM,25128,Essendon,91,Hawthorn,92,0.0


#### 2. Extract status.csv for AFL team player's Performance Report

In [23]:
stats_file = "Resources/stats.csv"
stats_df = pd.read_csv(stats_file)
stats_df.head()

,gameId,team,year,round,playerId,displayName,gameNumber,Disposals,Kicks,Marks,...,Brownlow Votes,Contested Possessions,Uncontested Possessions,Contested Marks,Marks Inside 50,One Percenters,Bounces,Goal Assists,% Played,Subs
0,2021R104,Adelaide,2021,R1,2021661124,"Berry, Sam",1,8,6,1,...,0,4,5,0,0,0,0,1,80,-
1,2021R104,Adelaide,2021,R1,2012662083,"Brown, Luke",168,5,2,0,...,0,2,3,0,0,0,0,0,23,Off
2,2021R104,Adelaide,2021,R1,2020665315,"Butts, Jordon",3,10,5,3,...,0,5,5,1,0,8,0,0,93,-
3,2021R104,Adelaide,2021,R1,2018689604,"Doedee, Tom",31,13,9,4,...,0,8,6,0,0,7,0,0,84,-
4,2021R104,Adelaide,2021,R1,2018703883,"Frampton, Billy",9,14,10,8,...,0,5,9,3,4,2,0,0,90,-


#### 3. Extract the sheet1 of AFL_Stadiums.xlsx for Venues

In [24]:
venue_file = "Resources/AFL_Stadiums.xlsx"
venue_df = pd.read_excel(io=venue_file,sheet_name=0,header=3)
venue_df.columns=['venue_name', 'in_use', 'games', 'goals', 'behinds', 'points', 'ave_sore', 'over_100']
venue_df.head()

,venue_name,in_use,games,goals,behinds,points,ave_sore,over_100
0,M.C.G.,1897-2022,2984,76566,74346,533742,89.43,2106
1,Princes Park,1897-2005,1277,31318,33362,221270,86.64,854
2,Docklands,2000-2022,1021,28445,23440,194110,95.06,860
3,Victoria Park,1897-1999,880,19679,22139,140213,79.67,443
4,Junction Oval,1897-1984,734,15692,18378,112530,76.66,319


#### 4. Extract the sheet2 of AFL_Stadiums.xlsx for AFL Stadiums

In [25]:
stadium_file = "Resources/AFL_Stadiums.xlsx"
stadium_df = pd.read_excel(io=stadium_file,sheet_name=1,header=2)
stadium_df.columns=['name', 'city_name', 'state_name', 'capacity']
stadium_df.head()

,name,city_name,state_name,capacity
0,Adelaide Oval,Adelaide,South Australia,53500.0
1,Alberton Oval,Adelaide,South Australia,11000.0
2,Albury Sports Ground,Albury,New South Wales,8000.0
3,Allinsure Park,Queanbeyan,New South Wales,8000.0
4,Arden Street Oval,Melbourne,Victoria,4000.0


## Transform premise DataFrame

### 1. Transform stats DataFrame to player table

In [26]:
# Create a filtered dataframe from specific columns
player_cols = ["playerId", "displayName"]
player_transformed = stats_df[player_cols].copy()

# split First name and Last name from full name 
player_name = player_transformed['displayName'].str.split(',', expand=True)
player_transformed['first_name']=player_name[1]
player_transformed['last_name']=player_name[0]

player_transformed.drop('displayName',axis=1,inplace=True)

# Rename the column headers
player_transformed = player_transformed.rename(columns={"playerId": "player_id"})

# Clean the data by dropping duplicates and setting the index
player_transformed.drop_duplicates("player_id", inplace=True)
player_transformed.set_index("player_id", inplace=True)

player_transformed.head()

,first_name,last_name
player_id,,
2021661124,Sam,Berry
2012662083,Luke,Brown
2020665315,Jordon,Butts
2018689604,Tom,Doedee
2018703883,Billy,Frampton


### 2.Transform stats DataFrame to team table

In [27]:

# Create a filtered dataframe from specific columns "team"
team_transformed = stats_df["team"].copy()
team_transformed.drop_duplicates(inplace=True)

team_transformed = team_transformed.reset_index()

# set auto-increment ID as team_id
team_transformed['team_id'] = range(1,len(team_transformed)+1)
# Rename the column headers
team_transformed.rename(columns = {'team':'team_name'}, inplace = True)
# Clean the data by dropping duplicates and setting the index
team_transformed.set_index("team_id", inplace=True)
team_transformed.drop('index',axis=1,inplace=True)

team_transformed.head(20)

,team_name
team_id,
1,Adelaide
2,Brisbane Lions
3,Carlton
4,Collingwood
5,Essendon
6,Fremantle
7,Gold Coast
8,Geelong
9,Greater Western Sydney


### 3. Transform stadium DataFrame to city table

In [28]:
# Create a filtered dataframe from specific columns
city_cols = ['city_name', 'state_name']
city_transformed = stadium_df[city_cols].copy()

# Clean the data by dropping duplicates and setting the index
city_transformed.drop_duplicates("city_name", inplace=True)

#set auto-increment ID as city_id" column with range function by lenth 
city_transformed['city_id'] = range(1,len(city_transformed)+1)
#set index for "city_id"
city_transformed.set_index("city_id", inplace=True)

city_transformed.head()

,city_name,state_name
city_id,,
1,Adelaide,South Australia
2,Albury,New South Wales
3,Queanbeyan,New South Wales
4,Melbourne,Victoria
5,Ballarat,Victoria


### 4. Transform stadium DataFrame

In [29]:

#replacae the name for the same stadium with abbreviation
stadium_df.replace('Melbourne Cricket Ground','M.C.G.', inplace=True)
stadium_df.replace('Sydney Cricket Ground','S.C.G.', inplace=True)
stadium_df.replace('Jiangwan Stadium (CHN)','Jiangwan Stadium', inplace=True)

#get unique names with unique function and union with the stadium name in stadium df and venue df in the same list
stadium_name = set(game_df["venue"].unique().tolist()).union(set(stadium_df['name'].unique().tolist())).union(set(venue_df['venue_name'].unique().tolist()))

stadium_name = pd.DataFrame(list(stadium_name),columns=['name'])
stadium_name['stadium_id'] = range(1,len(stadium_name)+1)


venue_cols = ['venue_name', 'in_use']
venue = venue_df[venue_cols].copy()
venue.rename(columns = {'venue_name':'name'}, inplace = True)

# split the in_use data into start year data as int and end year data as int
venue['start_year'] = venue['in_use'].astype(str).str[0:4].astype(int)
venue['end_year'] = venue['in_use'].astype(str).str[-4:].astype(int)

#left join with dataframe 'stadium_name'
venue = pd.merge(stadium_name, venue, on='name', how='left')
# venue.fillna(0, inplace=True)
venue.sort_values('end_year', ascending=False, inplace=True)

venue.head()


,name,stadium_id,in_use,start_year,end_year
65,Cazaly's Stadium,66,2011-2022,2011.0,2022.0
119,Sydney Showground,120,2012-2022,2012.0,2022.0
64,Docklands,65,2000-2022,2000.0,2022.0
62,Carrara,63,1987-2022,1987.0,2022.0
153,Stadium Australia,154,2002-2022,2002.0,2022.0


In [30]:

stadium_transformed = pd.merge(stadium_df, venue, on='name', how='outer')


# to connect same "city_name" in city_transformed and "stadium_transformed" to get the city_id 

def get_city_id(x):
    city_rows = city_transformed.loc[city_transformed['city_name'] == x]
    if len(city_rows)>0: # if the len greater than 0 , return the index as the city_id
        return city_rows.index.values[0]
    else:
        return ''
stadium_transformed['city_id'] = stadium_transformed['city_name'].map(get_city_id)

#  set criteria as if "end_year" is < 2022,return the boolean value true or false the stadium is active

def is_stadium_active(x):
    if x < 2022:
        return False
    elif isnan(x):
        return False
    else:
        return True
stadium_transformed['active_ind'] = stadium_transformed['end_year'].map(is_stadium_active)#use map to link active_ind back to the end_year list from stadium_transformed


stadium_transformed = stadium_transformed[['stadium_id', 'name', 'city_id', 'start_year', 'end_year', 'capacity', 'active_ind']]

#drop the duplicates and set the index
stadium_transformed.drop_duplicates("stadium_id", inplace=True)
stadium_transformed.set_index("stadium_id", inplace=True)
stadium_transformed.sort_values(by=['end_year', 'city_id'], ascending=[False, True], inplace=True)
stadium_transformed.dropna(inplace=True)
stadium_transformed.head(50)

,name,city_id,start_year,end_year,capacity,active_ind
stadium_id,,,,,,
129,Adelaide Oval,1,2011.0,2022.0,53500.0,True
122,M.C.G.,4,1897.0,2022.0,100024.0,True
139,S.C.G.,6,1903.0,2022.0,48000.0,True
10,Gabba,11,1981.0,2022.0,41974.0,True
69,Manuka Oval,29,1998.0,2022.0,16000.0,True
110,Jiangwan Stadium,31,2017.0,2019.0,12000.0,False
60,Riverway Stadium,50,2019.0,2019.0,10000.0,False
5,Victoria Park,4,1897.0,1999.0,10000.0,False
84,Windy Hill,4,1922.0,1991.0,10000.0,False


### 5.Transform stats DataFrame adding player performance to Game Player Status table

In [31]:

# Create a filtered dataframe from specific columns
stats_cols = ["gameId", "team", "playerId", "Rebounds","Inside 50s","Clearances","Contested Possessions"]
stats_transformed = stats_df[stats_df['year'] >= 2018][stats_cols].copy()

# Rename the column headers
stats_transformed = stats_transformed.rename(columns={"gameId": "game_id",
                                                        "playerId": "player_id",
                                                        "Rebounds": "rebound",
                                                        "Clearances": "clearance",
                                                        "Inside 50s": "inside_50s",
                                                        "Contested Possessions": "contested_possessions",
                                                     })
# by loc function, to find the 'team_name' from team_transformed df and find its lenth of row and set the index as team_id
def get_team_id(x):
    team_rows = team_transformed.loc[team_transformed['team_name'] == x]
    if len(team_rows)>0:
        return team_rows.index.values[0]
    else:
        return ''
stats_transformed['team_id'] = stats_transformed['team'].map(get_team_id)

#set the performance criteria for each player adding sum of key criterias: rebounds, inside_50s, clearance, contested possessions
stats_transformed['performance'] = stats_transformed[["rebound","inside_50s","clearance","contested_possessions"]].sum(axis=1)

stats_transformed.drop('team', axis=1, inplace=True)
# Clean the data by dropping duplicates and setting the index
stats_transformed.drop_duplicates("game_id", inplace=True)
stats_transformed.set_index("game_id", inplace=True)

stats_transformed.tail()


,player_id,rebound,inside_50s,clearance,contested_possessions,team_id,performance
game_id,,,,,,,
2018R109,2012674793,0,1,1,7,16,9
2018R404,2016655002,0,2,0,2,16,4
2018R1302,2016655002,4,1,0,5,16,10
2018R207,2015662872,2,1,0,6,17,9
2018R1808,2016676071,0,0,0,2,17,2


### 6. Transform game DataFrame

In [32]:
#select the data from dataframe above 2018 as latest data, we skip data of 2021&2022 due to data incomplete in covid19 
game_transformed = game_df[game_df['year'] >= 2018]

# Rename the column headers
game_transformed = game_transformed.rename(columns={"gameId": "game_id",
                                                        "startTime": "start_time",
                                                        "homeTeamScore": "home_team_score",
                                                        "awayTeamScore": "away_team_score",
                                                        "rainfall": "rain_fall"
                                                     })
#use 'homeTeam' to map to connect with 'home_team_id'
game_transformed['home_team_id'] = game_transformed['homeTeam'].map(get_team_id)
game_transformed['away_team_id'] = game_transformed['awayTeam'].map(get_team_id)
game_transformed.drop('homeTeam', axis=1, inplace=True)
game_transformed.drop('awayTeam', axis=1, inplace=True)

#left merge the game_transformend df with the venue
game_transformed = pd.merge(game_transformed, venue, left_on='venue', right_on='name', how='left')
game_transformed.drop('venue', axis=1, inplace=True)
game_transformed.drop('start_year', axis=1, inplace=True)
game_transformed.drop('end_year', axis=1, inplace=True)
game_transformed.drop('in_use', axis=1, inplace=True)

#str type transfer to date type
game_transformed['date'] = pd.to_datetime(game_transformed['date'], format='%d-%b-%Y')
# str type transfered to 24-hr time type
game_transformed['start_time'] = pd.to_datetime(game_transformed['start_time']).dt.strftime('%H:%M')
# drop the duplicates and set index
game_transformed.drop_duplicates("game_id", inplace=True)
game_transformed.set_index("game_id", inplace=True)


game_transformed.head()

,year,round,date,start_time,attendance,home_team_score,away_team_score,rain_fall,home_team_id,away_team_id,name,stadium_id
game_id,,,,,,,,,,,,
2021R101,2021,R1,2021-03-18,19:25,49218,105,80,0.0,14,3,M.C.G.,122
2021R102,2021,R1,2021-03-19,19:50,46051,53,69,0.0,4,18,M.C.G.,122
2021R103,2021,R1,2021-03-20,13:45,21365,80,58,0.0,11,6,M.C.G.,122
2021R104,2021,R1,2021-03-20,16:05,26985,103,91,0.0,1,8,Adelaide Oval,129
2021R105,2021,R1,2021-03-20,19:25,25128,91,92,0.0,5,10,Docklands,65


### 7. Transform TV DataFrame with team_id, TVS, year 2020

In [33]:
TVS_df =pd.read_csv(stats_file)
df1=TVS_df
df1_2=df1.query("year == 2020")

In [34]:
df2=df1_2.groupby(['team'])['Rebounds','Inside 50s','Clearances','Contested Possessions'].sum()

C:\Users\godlo\AppData\Local\Temp/ipykernel_9036/2268916263.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2=df1_2.groupby(['team'])['Rebounds','Inside 50s','Clearances','Contested Possessions'].sum()


In [35]:
df3 = df2.sum(axis = 1)
df4=pd.DataFrame(df3)
df4.columns=['TVS']


TVS_transformed=df4.sort_values(['TVS'],ascending=False)
TVS_transformed

,TVS
team,
Geelong,4778
Richmond,4638
Port Adelaide,4482
Brisbane Lions,4308
Collingwood,4163
St Kilda,4129
Western Bulldogs,3962
West Coast,3920
Carlton,3864


In [36]:
player_transformed.to_csv('output/player_transformed_result.csv',index=0)
stadium_transformed.to_csv('output/stadium_transformed_result.csv',index=0)
city_transformed.to_csv('output/city_transformed_result.csv',index=0)
team_transformed.to_csv('output/team_transformed_result.csv',index=0)
game_transformed.to_csv('output/game_transformed_result.csv',index=0)
stats_transformed.to_csv('output/stats_transformed_result.csv',index=0)
TVS_transformed.to_csv('output/stats_transformed_result.csv',index=0)

## Create database connection

In [37]:
connection_string = "postgres:postgres@localhost:5432/AFLGame_db"
engine = create_engine(f'postgresql://{connection_string}')

ModuleNotFoundError: No module named 'psycopg2'

## Load DataFrames into database

In [ ]:
player_transformed.to_sql(name='player', con=engine, if_exists='append', index=True)
team_transformed.to_sql(name='team', con=engine, if_exists='append', index=True)
city_transformed.to_sql(name='city', con=engine, if_exists='append', index=True)
stadium_transformed.to_sql(name='stadium', con=engine, if_exists='append', index=True)
stats_transformed.to_sql(name='stats', con=engine, if_exists='append', index=True)
game_transformed.to_sql(name='game', con=engine, if_exists='append', index=True)

NameError: name 'engine' is not defined